In [1]:
import wfdb
import scipy
import os
import glob
import subprocess

In [2]:

DATA_ROOT = 'D:/ECG/processed/physionet2021'
PHYSIONET_PATH = 'D:/ECG/physionet.org/files/challenge-2021/1.0.3/training'

In [3]:
MANIFEST_PATH = f'{DATA_ROOT}/manifests'
FINE_TUNE_MANIFEST = f'{MANIFEST_PATH}/cinc'
TOTAL_MANIFEST = f'{MANIFEST_PATH}/total/train.tsv'
PRE_TRAINING_MANIFEST = f"{MANIFEST_PATH}/cmsc"

In [4]:
FAIRSEQ_SIG_DIR = 'C:/Users/david/Documents/PythonScripts/fairseq-signals'
CONVERT_TO_CLOCS_MANIFEST = f'{FAIRSEQ_SIG_DIR}/fairseq_signals/data/ecg/preprocess/convert_to_clocs_manifest.py'
PRE_TRAINING_CONFIG = f'{FAIRSEQ_SIG_DIR}/examples/w2v_cmsc/config/pretraining'
FINE_TUNING_CONFIG = f'{FAIRSEQ_SIG_DIR}/examples/w2v_cmsc/config/finetuning/ecg_transformer'

## Preprocessing

In [5]:
cmd = f'python {FAIRSEQ_SIG_DIR}/fairseq_signals/data/ecg/preprocess/preprocess_physionet2021.py \
        {PHYSIONET_PATH} \
        --dest {DATA_ROOT} \
        --subset "WFDB_Ga, WFDB_CPSC2018, WFDB_CPSC2018_2, WFDB_ChapmanShaoxing"\
        --workers 1'

In [6]:
result = subprocess.run(cmd, stdout=subprocess.PIPE, shell=True)
print(result.stdout.decode('utf-8'))

In [10]:
cmd = f'python {FAIRSEQ_SIG_DIR}/fairseq_signals/data/ecg/preprocess/manifest.py \
        {DATA_ROOT} \
        --dest {MANIFEST_PATH} \
        --subset "CPSC2018, CPSC2018_2, ChapmanShaoxing, Ga" \
        --combine_subsets Ga'

In [11]:
result = subprocess.run(cmd, stdout=subprocess.PIPE, shell=True)
print(result.stdout.decode('utf-8'))

## Pre-training

In [7]:
cmd = f"python {CONVERT_TO_CLOCS_MANIFEST} \
            {TOTAL_MANIFEST} \
            --dest {MANIFEST_PATH}"
print(cmd)

python C:/Users/david/Documents/PythonScripts/fairseq-signals/fairseq_signals/data/ecg/preprocess/convert_to_clocs_manifest.py             D:/ECG/processed/physionet2021/manifests/total/train.tsv             --dest D:/ECG/processed/physionet2021/manifests


In [8]:
result = subprocess.run(cmd, stdout=subprocess.PIPE, shell=True)
print(result.stdout.decode('utf-8'))

In [9]:
cmd = f"fairseq-hydra-train \
            task.data={PRE_TRAINING_MANIFEST} \
            --config-dir {PRE_TRAINING_CONFIG} \
            --config-name w2v_cmsc_rlm_small"

In [ ]:
result = subprocess.run(cmd, stdout=subprocess.PIPE, shell=True)
print(result.stdout.decode('utf-8'))

## Fine-tuning

In [7]:
PRETRAIN_CHECKPOINT = f'C:/Users/david/Documents/PythonScripts/ECGPretraining/outputs/2023-04-22/11-42-07/checkpoints/checkpoint100.pt'
WEIGHTS_PATH = f'{PHYSIONET_PATH}/weights.csv'

In [6]:
WEIGHTS_PATH

'D:/ECG/physionet.org/files/challenge-2021/1.0.3/training/weights.csv'

In [7]:
cmd = f"fairseq-hydra-train \
        task.data={FINE_TUNE_MANIFEST} \
        model.model_path={PRETRAIN_CHECKPOINT} \
        --config-dir {FINE_TUNING_CONFIG} \
        --config-name diagnosis"

In [ ]:
result = subprocess.run(cmd, stdout=subprocess.PIPE, shell=True)
print(result.stdout.decode('utf-8'))

In [5]:
FINETUNE_CHECKPOINT = f'C:/Users/david/Documents/PythonScripts/ECGPretraining/outputs/2023-04-29/16-42-15/checkpoints/checkpoint_last.pt'


In [8]:
cmd = f"fairseq-hydra-validate \
        common_eval.path={FINETUNE_CHECKPOINT} \
        task.data={FINE_TUNE_MANIFEST} \
        model.model_path={PRETRAIN_CHECKPOINT} \
        --config-dir {FINE_TUNING_CONFIG} \
        --config-name diagnosis"

In [9]:
result = subprocess.run(cmd, stdout=subprocess.PIPE, shell=True)
print(result.stdout.decode('utf-8'))

[2023-04-29 22:31:31,255][fairseq_cli.validate][INFO] - loading model from C:/Users/david/Documents/PythonScripts/ECGPretraining/outputs/2023-04-29/16-42-15/checkpoints/checkpoint_last.pt
[2023-04-29 22:31:32,897][fairseq_signals.models.transformer][INFO] - Loaded pre-trained model parameters from C:/Users/david/Documents/PythonScripts/ECGPretraining/outputs/2023-04-22/11-42-07/checkpoints/checkpoint100.pt
[2023-04-29 22:31:32,915][fairseq_signals.utils.checkpoint_utils][INFO] - Loaded a checkpoint in 1.66s
[2023-04-29 22:31:32,916][fairseq_cli.validate][INFO] - num. shared model params: 62,041,754 (num. trained: 62,041,754)
[2023-04-29 22:31:32,917][fairseq_cli.validate][INFO] - num. expert model params: 0 (num. trained: 0)
[2023-04-29 22:31:33,107][fairseq_cli.validate][INFO] - {'_name': None,
 'checkpoint': {'_name': None, 'save_dir': 'checkpoints', 'restore_file': 'checkpoint_last.pt', 'finetune_from_model': None, 'reset_dataloader': False, 'reset_lr_scheduler': False, 'reset_meter